In [1]:
# Superhero dataset
from google.colab import files
uploaded = files.upload()

Saving Marvel-graph.txt to Marvel-graph.txt
Saving Marvel-names.txt to Marvel-names.txt


In [2]:
# Installing Spark 3.1.1 with dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
print('JDK 8 installed')
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
print('Spark 3.0.1 installer downloaded')
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
print('Spark 3.1.1 installed')
!pip install -q findspark
print('findspark installed')
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"
print('Environment Variables Set')
import findspark
findspark.init()
print('Install Complete')

JDK 8 installed
Spark 3.0.1 installer downloaded
Spark 3.1.1 installed
findspark installed
Environment Variables Set
Install Complete


In [3]:
# Most Popular Superhero in Marvel dataset
from pyspark.sql import SparkSession, functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Create Spark Session
spark = SparkSession.builder.appName('MostPopularSuperhero').getOrCreate()
# Custom Schema
schema = StructType([
                     StructField('id',IntegerType(),True),
                     StructField('name',IntegerType(),True)
])

# Apply Custom Schema to Schema file and Load Dataset
names = spark.read.schema(schema).option('sep',' ').csv('Marvel-names.txt')
lines = spark.read.text('Marvel-graph.txt')

In [17]:
# Marvel-graph.txt(1st value: The Hero. 2nd value: Connected Heroes)
names.head()

DataFrame[name: int]

In [4]:
# Split value column on spaces and take 1st element as ID (5988 above) and
# Create connections column from rest of row but remove ID value (748 onwards)
connections = lines.withColumn('id',func.split(func.col('value'),'')[0]) \
.withColumn('connections',func.size(func.split(func.col('value'),' ')) -1) \
.groupBy('id').agg(func.sum('connections').alias('connections'))

In [8]:
# Getting Most Popular Hero
mostPopular = connections.sort(func.col('connections').desc()).first()
mostPopularName = names.filter(func.col('id') == mostPopular[0]).select('name').first()

In [10]:
mostPopularName[0]

In [7]:
print(mostPopularName[0] + ' with ' + str(mostPopular[1]) + ' connections is most popular')

TypeError: ignored

In [ ]:
# Stop session
spark.stop()